# Imports

In [ ]:
import numpy
import matplotlib.pyplot as plt
from tqdm import tqdm
from auxiliary_functions import *
import random
random.seed(0)
numpy.random.seed(0)

N = 500 # number of observation
p = 100 # dimension of observations
M = 100 # number of experiments per delta
BIAS = True # Whether to use the biased or unbiased covariance estimator
effective_rank = 5

# Cell Wise extreme outliers

In [ ]:
sigma, eigen = low_rank(5, 100, 500)
X = numpy.random.multivariate_normal(N, p, sigma)
X_noisy = contaminate_bernoulli(X, sigma, s=5, max_iter=1e3)

# Spectral adversarial attack

#### Proof of concept for the attack

In [ ]:
# Version with 5% perturbation
sigma, eigen = low_rank(5, 100, 500)
X = numpy.random.multivariate(N, p, sigma)
X_noisy = contaminate_adversarial(X, sigma, s=5, max_iter=1e3)

print("fro norm of error of cov estimator :                            ", numpy.linalg.norm(numpy.cov(X.T) - sigma))
print("fro norm of error of cov estimator after adverse perturbation : ", numpy.linalg.norm(numpy.cov(X_noisy.T) - sigma))

In [ ]:
# Comparison of the first 10 eigenvalues of the true covariance matrix and
# the estimated covariance matrices with and without the adversarial
# perturbation. Ideally, the differences should be almost invisible,
# showing that this kind of adversarial attack isn't trivially identifiable.
u, d, v = numpy.linalg.svd(sigma)
u_hat, d_hat, v_hat = numpy.linalg.svd(numpy.cov(X.T))
u_noise, d_noise, v_noise = numpy.linalg.svd(numpy.cov(X_noisy.T))

plt.figure(figsize=(10,8))
plt.plot(d[:10])
plt.plot(d_hat[:10])
plt.plot(d_noise[:10])
plt.legend(["True", "estimated", "estimated and perturbed"])
plt.title("comparison of first 10 eigenvalues")
plt.show()

In [ ]:
# Comparison of the error in estimated eigenspaces, by computing the frobenius
# distance between projectors. Although the eigenvalues remain very similar
# after the attack, the eigenspaces change, thus proving the attack alters
# the overall structure of the covariance matrix.
print("fro norm error of first eigenvector : ", numpy.linalg.norm(v_hat[0] - v[0]))
print("same with perturbation              : ", numpy.linalg.norm(v_noise[0] - v[0]))

dists_hat = []
dists_noise = []
for i in range(len(d)):
    # thetas designate projectors to the eigenspace
    theta_hat = numpy.outer(v_hat[i], v_hat[i].T)
    theta = numpy.outer(v[i], v[i].T)
    theta_noise = numpy.outer(v_noise[i], v_noise[i].T)
    error_noise = numpy.linalg.norm(theta_noise[i] - theta[i])
    error_hat = numpy.linalg.norm(theta_hat[i] - theta[i])
    dists_hat.append(error_hat)
    dists_noise.append(error_noise)

plt.plot(dists_hat)
plt.plot(dists_noise)
plt.legend(["fro error of eigenvectors", "fro error of perturbed estimated eigenvectors"])
plt.title("Error of estimated eigenvectors with and without perturbation")
plt.show()

#### Comparison of different estimators